0. Open
1. Review
2. Topic

    2.1 순환 신경망(Recurrent Neural Network, RNN)
    
    2.2 장단기 메모리(Long Short - Term Memory, LSTM)
    
3. Q & a
4. Next
5. Close

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add( Dense( 1, input_dim = 3, activation = 'relu' ) )

In [ ]:
model.summary()

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

model = Sequential()
model.add(Dense( 8, input_dim = 4, activation = 'relu' ) )
model.add(Dense( 1, activation = 'sigmoid' ) ) # 출력층

In [ ]:
model.summary()

## 순환 신경망(Recurrent Neural Network, RNN)

- RNN은 시퀀스(Sequence) 모델
- 입력과 출력을 스퀀스 단위로 처리하는 인공 신경망 모델
- 자연어 처리에서 문장을 단어 시퀀스라 표현
- 인공 신경망에서 시퀀스 처리를 위한 모델 - RNN

- 피드 포워드 신경망(FFNN) : 입력층에서 은닉층을 거쳐서 출력층으로 학습하는 방법, 한 방향으로 학습
- 순환 신경망(RNN) : 입력층에서 은닉층을 거쳐서 출력층으로 학습하는 방법, 은닉층에 출력이 다음 층으로 전달되는 것 외에 다시 해당 은닉층으로 입력되는 특성

- 은닉층의 메모리 셀은 각각의 시점(time step)에서 바로 이전 시점에서의 인닉층의 메모리 셀에서 나온 값을 자신의 입력값으로 사용하는 재귀적 활동을 한다.
- 은닉층으로 입력되는 값은 2개 : 이전 층의 출력값, ***이전 시점의 은닉층의 출력값(이전 시점의 값)***
- 셀은 출력층 방향으로 또는 다음 시점 t + 1의 자신에게 보내는 값을 은닉 상태(hidden state)라고 한다.
    - t - 1 : 이전 셀의 출력값
    - t : 현재 셀의 상태값
    - t + 1 : 다음 시점의 값

- 하나의 입력에 대해서 여러개의 출력(one-to-many) : 이미지에 대한 캡션 추가(시퀀스 출력)
- 단어 시퀀스에 대해서 하나의 출력(many-to-one) : 영화평에 대한 긍정/부정 판별(감성 분류), 스팸메일 판별
- 입력 문장으로부터 대답 문장 출력(many-to-many) : 번역, 챗봇

- RNN 내부 동작(식)

* ht : 현재 시점에서 t의 은닉 상태값
* ht-1 : 이전 시점 t - 1의 은닉 상태값
* 은닉층의 메모리 셀은 ht를 계산하기 위해서 총 두 개의 가중치를 갖는다.
* Wx는 입력값 Xt에 대한 가중치, Wh는 이전 시점 t - 1의 은닉상태값 ht - 1에 대한 가중치

- 은닉층 : ht = tanh(WxXt + Whht-1 + b)
- 출력층 : yt = f(Wyht + b) -> f : 비선형 활성화 함수

### RNN 은닉층 연산을 벡터와 행렬 연산으로 이해
- 은닉층 (은닉 상태) : ht = tanh(Whht-1 + WxXt + b)
- 자연어 처리에서 RNN의 입력 Xt는 대부분의 경우 단어 벡터로 간주
- 단어 벡터의 차원을 d로 하고, 은닉 상태의 크기를 Dh라 할 때 벡터와 행렬의 크기
    - Xt : (d X 1) : 입력 층
    - Wx : (Dh X d) : 입력 층에 대한 가중치
    - Wh : (Dh X Dh) : 이전 시점에 대한 가중치
    - ht-1 : (Dh X 1) : 이전 시점의 은닉 상태 
    - b : (Dh X 1) : 편향 (bias)

### 은닉층 (은닉 상태) : ht = tanh(Whht-1 + WxXt + b)를 파이썬으로 구현

- pseudo code

```
hidden_state = 0 # 초기 은닉 상태, 0으로 초기화
for input_t in input_length # 각 시점마다 입력
    output_t = tanh(input_t, hidden_state_t) # 각 시점에 대한 은닉 상태 연산
    hidden_state_t = output_t # 현재 시점의 은닉상태
``` 

In [ ]:
import numpy as np

timesteps = 10 # 시점의 수 , NLP 관점에서는 문장의 길이
input_dim = 4 # 입력 차원, NLP 관점에서는 단어 벡터의 차원
hidden_size = 8 # 은닉 상태의 크기, 메모리 셀의 용량

inputs = np.random.random((timesteps, input_dim)) # 입력에 해당하는 2D 텐서
hidden_state_t = np.zeros((hidden_size, )) # 초기 은닉 상태

In [ ]:
print(hidden_state_t)

In [ ]:
Wx = np.random.random((hidden_size, input_dim))
Wh = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size))

In [ ]:
print(Wx.shape)

In [ ]:
total_hidden_states = []

# 메모리 셀 동작
for input_t in inputs: # 각 시점에 따라서 입력값 입력
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b) # Wx*Xt + Wh*ht-1 + 1
    total_hidden_states.append(list(output_t)) # 각 시점 은닉 상태값 축적
    print(np.shape(total_hidden_states)) # 각 시점 t별 메모리 셀의 크기
    hidden_state_t = output_t
    
total_hidden_states = np.stack(total_hidden_states, axis=0)
print(total_hidden_states) # (timesteps, output_dim) 2D텐서 출력

### Keras로 RNN 구현

In [ ]:
# 가장 간단한 RNN 표현

model.add(SimpleRNN(hidden_size))
model.add(SimpleRNN(hidden_size, input_shape=(timesteps, input_dim)))
model.add(SimpleRNN(hidden_size, input_length=M, input_dim=N))

- hidden_size : 은닉 상태의 크기, 메모리 셀이 다음 시점의 메모리 셀과 출력층으로 보내는 값의 크기(output_dim)와 동일, RNN의 용량(capacity)를 늘린다고 보면 되며, 중소형 모델의 경우 보통 128, 256, 512, 1024등의 값을 가짐
- timesteps : 입력 시퀀스의 길이(input_length)라고 표현, 시점의 수
- input_dim = 입력의 크기



- timestep = 3일 때, return_sequences = True를 설정한 경우와 그렇지 않은 경우
- return_sequences = True -> 메모리 셀이 모든 시점(time step)에 대해서 은닉 상태값을 표시
- return_sequences = False -> 메모리 셀은 하나의 은닉 상태값만 출력한다.

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN

In [ ]:
model = Sequential()

model.add(SimpleRNN(3, input_shape=(2, 10)))

model.summary()

In [ ]:
model = Sequential()

model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10)))

model.summary()

In [ ]:
model = Sequential()

model.add(SimpleRNN(3, batch_input_shape=(8, 2, 10), return_sequences=True))

model.summary()

### 깊은 순환 신경망(Deep Recurrent Neural Network)



In [ ]:
model = Sequential()

model.add(SimpleRNN(hidden_size, return_sequence=True))
model.add(SimpleRNN(hidden_size, return_sequence=True))

### 양방향 순환 신경망(Bidirectional Recurrent Neural Network)

In [ ]:
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequence=True), input_shape=(timesteps, input_dim)))

### 깊은 양방향 순환 신경망(Deep Bidirectional Recurrent Neural Network)

In [ ]:
# 은닉층이 4개인 경우
model = Sequential()
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequence=True), input_shape=(timesteps, input_dim)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequence=True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequence=True)))
model.add(Bidirectional(SimpleRNN(hidden_size, return_sequence=True)))

### 바닐라 RNN의 한계

- 바닐라 RNN의 출력 결과가 이전의 계산 결과에 의존
- 바닐라 RNN은 비교적 짧은 시퀀스에 대해서만 효과를 보이는 단점
- 바닐라 RNN은 시점(time step)이 길어질수록 앞의 정보가 뒤로 충분히 전달되지 못하는 현상이 발생한다.
- 장기 의존성 문제(the problem of Long-term dependencies)
- 장기 의존성 문제를 해결하기 위한 RNN 변경이 장단기 메모리(LSTM)
- LSTM과 바닐라 RNN과의 차이는 입력/삭제/출력 게이트와 셀상태라는 입력값이 LSTM에 추가

- LSTM은 은닉 상태값과 셀 상태값을 구하기 위해서 3개의 게이트를 사용한다.
- 각 게이트는 입력 게이트, 삭제 게이트, 출력 게이트라 부르고, 공통적으로 시그모이드(sigmoid) 함수를 사용한다.

### 입력 게이트
- it = sigmoid(WxiXi + Whiht-1 + bi)  
- gt = tanh(WxgXt + Whght-1 + bg)

- 현재 정보를 기억하기 위한 게이트

### 삭제 게이트

- 기억을 삭제하기 위한 게이트
- ft = sigmoid(Wxf * xt + Whf * ht-1 + bf)

### 출력 게이트
- Dt = sigmoid(Wxo * xt + Who * ht-1 + bo)
- ht = Ot . tanh(ct)
- 출력 게이트는 현재 시점의 t의 x값과 이전 시점 t-1의 은닉 상태가 시그모이드 함수를 지난 값이고, 해당 값은 시점 t의 은닉 상태를 결정하는 일을 한다.
- 은닉 상태는 단기 상태라고도 한다.

### 셀 상태(장기 상태)

- Ct = ft . Ct-1 + it . gt
- Ct : 셀 상태
- it, gt : 입력 게이트에서 계산된 값
- ft : 삭제 게이트에서 계산된 값
- . : 원소별 곱(entrywise product)

- 게이트 순환 유닛(Gated Recurrent Unit, GRU) : LSTM에 비해서 은닉상태 계산을 줄인 모델
- 업데이트 게이트와 리셋 게이트로 구성

- 1D Conv 모델 : CNN 계열 모델